In [1]:
from PIL import Image
import numpy as np
import pyresample as pr
from pycoast import ContourWriterAGG
import aggdraw

In [2]:
Source_def = pr.load_area('Table/Projection.proj','KoChi_1800') # 原始FDK資料的投影
Image_def = pr.load_area('Table/Projection.proj','TW_800')  # 輸出圖檔的投影

In [24]:
def img2lcc(imgS):
    global Source_def, Image_def
    cntS=np.array(imgS)
    RoI = 5000 # 取樣之影響半徑
    Si = 25000
    #cntLCC = pr.kd_tree.resample_nearest(Source_def, cntS, Image_def, radius_of_influence=RoI)
    cntLCC = pr.kd_tree.resample_gauss(Source_def, cntS, Image_def, 
                                       radius_of_influence=RoI, 
                                       sigmas=Si)
    return Image.fromarray(np.uint8(cntLCC))
    

In [25]:
def mergeImage(img1,img2):
    global Image_def
    imgNew = Image.merge('RGB',(img1,img1,img2))
    ShapePath = 'Table/gshhg-shp-2.3.7'
    cw = ContourWriterAGG(ShapePath)
    cw.add_coastlines(imgNew,Image_def,resolution='i'
                      ,outline=(255,255,255),width=2)
    ttfFile = 'Table/LiberationMono-Regular.ttf'
    font = aggdraw.Font('white',ttfFile,size=30)
    cw.add_grid(imgNew,Image_def,(5,5),(5,5),
                font,outline=(250,250,255),width=1)
    return imgNew

In [26]:
if __name__ == '__main__':
    IR1 = Image.open('HMW823080103IR1.pgm')
    VIS = Image.open('HMW823080103VIS.pgm')
    IR1LCC = img2lcc(IR1)
    VISLCC = img2lcc(VIS)
    img = mergeImage(VISLCC,IR1LCC)
    img.show()
    img.save('gauss.jpg')